<a href="https://colab.research.google.com/github/jpalomogz/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/JavierPalomo_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre: Javier Palomo Gonzalez <br>
Link: https://colab.research.google.com/drive/1V217xGzKZMSR-3fSQ1M4pgO5s_zCbDEW<br>
Actividad Guiada 2

# **Programacion Dinamica**

In [0]:
matrizTarifas = [
[0,5,4,3,999,999,999],      # desde nodo 0
[999,0,999,2,3,999,11],     # desde nodo 1
[999,999, 0,1,999,4,10],    # desde nodo 2
[999,999,999, 0,5,6,9],     # desde nodo 3
[999,999, 999,999,0,999,4], # desde nodo 4
[999,999, 999,999,999,0,3], # desde nodo 5
[999,999,999,999,999,999,0] # desde nodo 6
]

In [0]:
def Precios(tarifa):

  N = len(tarifa[0])

  #Inicializamos dos matrices NxN para almacenar precios y la ruta
  precio = [[9999]*N for i in [9999]*N]
  ruta = [[""]*N for i in [""]*N]

  for i in range (N-1):
    for j in range(i+1,N):
      tarifaMinima = tarifa[i][j]
      ruta[i][j] = i

      for k in range(i,j):
        if precio[i][k] + tarifa[k][j] < tarifaMinima:
          tarifaMinima = min(tarifaMinima,precio[i][k] + tarifa[k][j])
          ruta[i][j] = k
        precio[i][j] = tarifaMinima

  return precio, ruta

In [0]:
PRECIOS,RUTA = Precios(matrizTarifas)  
#print(PRECIOS[0][6])

print("PRECIOS")
for i in range(len(matrizTarifas)):
  print(PRECIOS[i])

print("\nRUTA")  
for i in range(len(matrizTarifas)):
  print(RUTA[i]) 

PRECIOS
[9999, 5, 4, 3, 8, 8, 11]
[9999, 9999, 999, 2, 3, 8, 7]
[9999, 9999, 9999, 1, 6, 4, 7]
[9999, 9999, 9999, 9999, 5, 6, 9]
[9999, 9999, 9999, 9999, 9999, 999, 4]
[9999, 9999, 9999, 9999, 9999, 9999, 3]
[9999, 9999, 9999, 9999, 9999, 9999, 9999]

RUTA
['', 0, 0, 0, 1, 2, 5]
['', '', 1, 1, 1, 3, 4]
['', '', '', 2, 3, 2, 5]
['', '', '', '', 3, 3, 3]
['', '', '', '', '', 4, 4]
['', '', '', '', '', '', 5]
['', '', '', '', '', '', '']


In [0]:
def calcular_ruta(ruta, desde, hasta):
  if desde == hasta:
    #print("Ir a :" + str(desde))
    return desde 
  else:
    return str(calcular_ruta(ruta, desde, ruta[desde][hasta])) +  ',' + str(ruta[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(RUTA, 0,6) 


La ruta es:


'0,0,2,5'

# **Ramificacion y Poda**

In [0]:
# Problema de Asignacion de tareas
costes=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [0]:
def valor(s,costes):
  valor = 0
  for i in range(len(s)):
    valor += costes[s[i]][i] 
  return valor
  
valor((0, 1, 2, 3),costes)

73

In [0]:
import itertools

In [0]:
#Fuerza Bruta
def fuerza_bruta(costes):
  mejor_valor = 10e100
  mejor_solucion = ()

  for s in list(itertools.permutations(range(len(costes)))):
    valor_tmp = valor(s,costes)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = s
  
  print("La mejor solucion es: ", mejor_solucion," con valor:", mejor_valor)

fuerza_bruta(costes)

La mejor solucion es:  (0, 3, 1, 2)  con valor: 61


In [0]:
# Coste inferior para soluciones parciales
# (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(s,costes):
  valor = 0
  #Valores establecidos
  for i in range(len(s)):
    valor += costes[i][s[i]]

  #Estimacion
  for i in range(len(costes)):
    if i not in s:
      valor += min([costes[j][i] for j in range(len(s),len(costes))])
  return valor

In [0]:
#Genera tantos hijos como como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(nodo, n):
  hijos = []
  for i in range(n):
    if i not in nodo:
      hijos.append({'s':nodo +(i,)})
  return hijos

In [0]:
def CS(s,costes):
  valor = 0
  #Valores establecidos
  for i in range(len(s)):
    valor += costes[i][s[i]]

  #Estimacion
  for i in range(len(costes)):
    if i not in s:
      valor += max( [costes[j][i] for j in range(len(S), len(costes)) ])
  return valor

In [0]:
def ramificacion_y_poda(costes):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(costes)
  dimension = len(costes)  
  mejor_solucion=tuple( i for i in range(len(costes)) )
  CotaSup = valor(mejor_solucion,costes)
  #print("Cota Superior:", CotaSup)

  nodos=[]
  nodos.append({'s':(),'ci':CI((),costes)} )

  iteracion = 0

  while( len(nodos) > 0):
    iteracion +=1

    nodo_prometedor = [ min(nodos, key=lambda x:x['ci'])][0]['s']
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    hijos = [{'s':x['s'],'ci':CI(x['s'], costes)} for x in crear_hijos(nodo_prometedor, dimension)]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    nodo_final = [x for x in hijos if len(x['s']) == dimension]
    if len(nodo_final) > 0: 
      #print("\n********Soluciones:",  [x for x in hijos if len(x['s']) == dimension])
      if nodo_final[0]['ci'] < CotaSup:
        CotaSup = nodo_final[0]['ci']
        mejor_solucion = nodo_final
 
    #Poda
    hijos = [x for x in hijos if x['ci'] < CotaSup]

    #Añadimos los hijos 
    nodos.extend(hijos) 

    #Eliminamos el nodo ramificado
    nodos =  [x for x in nodos if x['s'] != nodo_prometedor]
   
    print("La solucion final es:" ,mejor_solucion , " en " , iteracion , " iteraciones" , " para dimension: " ,dimension)
      


ramificacion_y_poda(costes)

La solucion final es: (0, 1, 2, 3)  en  1  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  2  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  3  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  4  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  5  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  6  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  7  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  8  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  9  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  10  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  11  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  